In [1]:
import numpy as np
import pandas as pd
from scipy.linalg import cholesky
%matplotlib ipympl

In [2]:
from src.randmixtw import randmixtw
from src.idx import idx
from src.margX import margX
from src.pllik_obs import pllik_obs
from src.EMtw import EMtw

# SCR pour les variables ordinales

Ce notebook se base sur l'article [*A MODEL-BASED APPROACH TO SIMULTANEOUS CLUSTERING AND DIMENSIONAL REDUCTION OF ORDINAL DATA*](https://ecampus.paris-saclay.fr/pluginfile.php/3267823/mod_folder/content/0/articles/Ranalli-2017-s11336-017-9578-5.pdf) écrit par M. Ranalli et R. Rocci. 

Cet article propose une méthode de clustering et de réduction de dimension (SCR), pour des variables ordinales, basées sur un algotithme EM-Pairwise.

## Simulation de données 

In [3]:
# Paramètres d'entrée
np.random.seed(123)
N, P, G = 500, 4, 3

In [4]:
# Génération des paramètres de mélange
Mu = [np.zeros(P), 2 + np.random.randn(P), 4 + np.random.randn(P)] 
Sigma = []
## Paramètres pour la composante de bruit du mélange
S = np.random.randn(P, P)
A = np.linalg.cholesky(S.T @ S)
np.fill_diagonal(A, 1)
Sigma.append(A.T @ A)
## Paramètres pour les composantes discriminantes du mélange
for g in range(1,G):
    S = np.random.randn(P, P) # Génération d'une matrice de taille P*P
    Sigma.append(S.T @ S) # Création de la matrice de variance covariance

pg = [0.2, 0.3, 0.5] # répartition du mélange
no_TH = np.full(P, 5)

# Génération du mélange gaussien
outx = randmixtw(N, Mu, Sigma, pg, no_TH)

In [5]:
# Création de l'espace des paramètres theta

# Calcul des matrices de Cholesky des matrices Sigma
a = []
for g in range(G):
    a.append(cholesky(Sigma[g]))
  
# Paramètres du modèle
theta = {
    "ts": outx["ts"],  # Seuils
    "Mu": Mu,          # Moyennes
    "Sigma": Sigma,     # Matrices de covariance
    "a" : a
}  

In [6]:
# Calcul des marginales
mX = margX(outx['X'], no_TH)

# Calcul des indices
Ind = idx(P, mX, no_TH)

# Calcul de la log vraissemblance observée
pll = pllik_obs(theta, pg, Ind, P, G)
print("La log-vraisemblance observée est : ", pll)

La log-vraisemblance observée est :  -3933.9667786440145


## Test de l'algorithme dans le cas où les données initials sont les vrais données

In [7]:
out_1 = EMtw(theta, pg, mX, Ind, G, P, no_TH, tol = 1e-4)

print("pg ", out_1["pg"])
print("\nmu ", out_1["mu"])
print("\nSigma ", out_1["sigma"])

|-------------|-------------|-------------|-------------|
|     iter    |    imp      |     lik     |  llik-llko  |
|-------------|-------------|-------------|-------------|
          1 |     5084.23 |     -3170.1 |         inf
          2 |     3677.24 |    -3267.73 |    -97.6274
|-------------|-------------|-------------|-------------|
pg  [0.45719509 0.27821381 0.2645911 ]

mu  [[ 0.          0.          0.          0.        ]
 [ 0.63759347  2.52079684  0.42504077 -3.58428218]
 [ 0.40530901  1.680585   -0.30656033  2.37746121]]

Sigma  [array([[ 1.        ,  3.65210121, -2.00374012,  0.30990031],
       [ 3.65210121, 14.33784326, -7.74434092, -0.51294452],
       [-2.00374012, -7.74434092,  5.19685897, -0.16737261],
       [ 0.30990031, -0.51294452, -0.16737261,  3.86261399]]), array([[ 4.98555435,  5.70477973, -1.73604616,  4.90508304],
       [ 5.70477973, 17.48022737, -1.73905228, -3.21723752],
       [-1.73604616, -1.73905228,  9.77766034, -1.44741206],
       [ 4.90508304, -3.

In [9]:
print("pg ", [0.2, 0.3, 0.5])
print("\nmu ", theta["Mu"])
print("\nSigma ", theta["Sigma"])

pg  [0.2, 0.3, 0.5]

mu  [array([0., 0., 0., 0.]), array([0.9143694 , 2.99734545, 2.2829785 , 0.49370529]), array([3.42139975, 5.65143654, 1.57332076, 3.57108737])]

Sigma  [array([[ 2.7219606 ,  1.58814553, -0.23774124,  0.3129903 ],
       [ 1.58814553,  2.42773383, -0.10968452,  1.00030977],
       [-0.23774124, -0.10968452,  1.58215056, -0.76298792],
       [ 0.3129903 ,  1.00030977, -0.76298792,  1.        ]]), array([[ 4.73017361,  2.1621835 , -2.74713088,  2.49179844],
       [ 2.1621835 ,  2.11282712, -1.61265319,  3.63956355],
       [-2.74713088, -1.61265319,  2.52195184, -0.99109121],
       [ 2.49179844,  3.63956355, -0.99109121,  9.98382333]]), array([[ 4.76171427,  2.18636121, -0.0899626 ,  0.52556341],
       [ 2.18636121,  8.63117157, -2.27299474, -4.17456998],
       [-0.0899626 , -2.27299474,  7.55175213,  2.13955745],
       [ 0.52556341, -4.17456998,  2.13955745,  2.72226721]])]


## Test de l'algorithme dans le cas de données prises aléatoirement 

In [21]:
np.random.seed(4)
N, P, G = 500, 4, 3

# initialisation des poids
pg = np.random.rand(G)
pg /= np.sum(pg)

# initialisation des moyennes et variances
mu = []
sigma = []
# ts = [np.array([-100000, -40, 0, 20, 80, 10000])/2]
a=[]
for g in range(G):
    mu.append(np.random.rand(P)*2)
    S = np.random.randn(P,P)*2
    sigma.append(S.T@S)
    # ts.append(np.array([-100000, -40, 0, 20, 80, 10000])/2)
    a.append(cholesky(sigma[g]))
    
theta0 = {
    "ts": theta["ts"],  # Seuils
    "Mu": mu,          # Moyennes
    "Sigma": sigma,     # Matrices de covariance
    "a" : a
}

In [23]:
print(theta0["ts"])

[array([-1.00000000e+05, -3.11501633e+00, -8.49713251e-01,  1.41558983e+00,
        3.68089291e+00,  1.00000000e+05]), array([-1.00000000e+05, -1.07745909e+00,  2.04330865e+00,  5.16407638e+00,
        8.28484412e+00,  1.00000000e+05]), array([-1.00000000e+05, -1.96949552e+00,  3.75687927e-01,  2.72087137e+00,
        5.06605482e+00,  1.00000000e+05]), array([-1.00000000e+05, -8.66188547e-01,  1.84512261e+00,  4.55643377e+00,
        7.26774493e+00,  1.00000000e+05])]


Il manque à calculer les probabilités a posteriori pour chaque individu. Ici, notre algorithme EM-pairwise nous donne ces probabilités pour les couples de réponses. 

In [24]:
out_2 = EMtw(theta0, pg, mX, Ind, G, P, no_TH, tol = 1e-4)

print("pg ", out_1["pg"])
print("\nmu ", out_1["mu"])
print("\nSigma ", out_1["sigma"])

|-------------|-------------|-------------|-------------|
|     iter    |    imp      |     lik     |  llik-llko  |
|-------------|-------------|-------------|-------------|
          1 |     3764.87 |    -3968.21 |         inf
          2 |     3680.18 |    -3668.31 |     299.905
          3 |     2024.61 |    -3561.03 |      107.28
          4 |     1830.66 |    -3568.42 |     17.2671
          5 |     3075.38 |    -3604.66 |     68.9714
          6 |     3650.14 |    -3549.31 |      116.47
          7 |     3514.63 |    -3843.66 |     307.914
          8 |      7718.3 |    -3661.31 |     117.819
          9 |     819.675 |    -4030.36 |     171.196
         10 |       732.9 |    -3792.94 |     75.9851
         11 |      581.64 |    -3981.75 |     65.2152
         12 |     574.978 |    -3839.02 |     44.4712
         13 |     617.287 |    -3835.99 |     1.24017
         14 |     672.896 |    -3785.68 |     23.1907
         15 |     1189.17 |    -3823.05 |      17.716
         16 |   

/Users/enzob/Desktop/études/M2 MIA/Periode 1&2/NSA/article/src/parl2v.py:31: RuntimeWarning: invalid value encountered in log
  param0 = np.concatenate([param0, [ts_clean[0]], np.log(np.diff(ts_clean))])


ValueError: array must not contain infs or NaNs